In [ ]:
#bbknn run - Peng He
import numpy as np
import Scanpyplus
import pandas as pd
import scanpy as sc
import anndata
import bbknn

In [ ]:
#define sample names from limb and bone marrow
library=['5386STDY7537944','5478STDY7819848','5478STDY7850147','WSSS_THYst8796439',
'5386STDY7557335','5478STDY7819849','5478STDY7850148','WSSS_THYst8796440',
'5386STDY7557336','5478STDY7840021','5478STDY7850149','WSSS_THYst8796441',
'5386STDY7557337','5478STDY7840022','5478STDY7935101','WSSS_THYst8796442',
'5478STDY7652318','5478STDY7840023','5478STDY7935102','WSSS_THYst9384953',
'5478STDY7717491','5478STDY7840024','5478STDY7980348','WSSS_THYst9384954',
'5478STDY7717492','5478STDY7850142','5478STDY7980349','WSSS_THYst9384955',
'5478STDY7819844','5478STDY7850143','FCAImmP7536758','WSSS_THYst9384956',
'5478STDY7819845','5478STDY7850144','FCAImmP7536759','WSSS_THYst9384957',
'5478STDY7819846','5478STDY7850145','WSSS_THYst8796437','WSSS_THYst9384958',
'5478STDY7819847','5478STDY7850146','WSSS_THYst8796438']
library2=['FCAImmP7179368','FCAImmP7277567','FCAImmP7555854','FCAImmP7803019','FCAImmP7803033',
'FCAImmP7277559','FCAImmP7292031','FCAImmP7555855','FCAImmP7803031',
'FCAImmP7277566','FCAImmP7292035','FCAImmP7803018','FCAImmP7803032']

In [ ]:
#read in the SoupX adjusted matrices
ad=[]
for i in library:
    temp=sc.read_10x_mtx('results/'+i+'/output/Gene/SoupX/')
    ad.append(temp)
for i in library2:
    temp=sc.read_10x_mtx('results2/'+i+'/output/Gene/SoupX/')
    ad.append(temp)

In [ ]:
#concatenate
adata = ad[0].concatenate(ad[1:], batch_categories=library+library2)

In [43]:
#add meta data and mito genes
Meta=pd.read_csv('Meta.csv',index_col=0)
adata=Scanpyplus.AddMetaBatch(adata,Meta)
mito_genes=adata.var_names[adata.var_names.str.startswith('MT-')]
adata.obs['percent_mito']=np.sum(adata[:,
                        mito_genes].X,axis=1).A1 / np.sum(adata.X, axis=1).A1
adata= adata[adata.obs['percent_mito'] < 0.1, :]

In [ ]:
#normalise, scale, filter and run bbknn
sc.pp.normalize_per_cell(adata,counts_per_cell_after= 1e4, key_n_counts= 'n_counts')
sc.pp.log1p(adata)
sc.pp.scale(adata)
sc.tl.pca(adata)
sc.pp.filter_cells(adata,min_genes=500)
bbknn.bbknn(adata,batch_key='batch',
            neighbors_within_batch = 3, trim = 200,
            approx=False,metric='euclidean',n_pcs=100)

In [195]:
#save h5ad for conversion in seurat
adata.write_h5ad('Cartilage.h5ad')